# Importing

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ee
import geemap
import fiona
import time
import json
from pathlib import Path
import datetime
import numpy as np
from utils.gee import export_sar, export_opt, export_maskcloud, export_opt_l1
import yaml

# Authentication

In [3]:
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


# Site Selection

In [118]:
site = 1
stie_cfg = Path('sites') / f'site_{site}.yaml'

In [119]:
with open(stie_cfg, 'r') as file:
    cfg = yaml.safe_load(file)

In [114]:
cfg

{'xcenter': -59.98953959,
 'ycenter': -7.10765019,
 'xmin': -60.55908023,
 'ymin': -7.65037612,
 'xmax': -59.41999894,
 'ymax': -6.56492426,
 'date': ['2019-08-14', '2020-07-31', '2021-07-02'],
 'n_sar_avg_images': 12,
 'before_days': 30,
 'opt_images_dates': [['2019-08-06', '2019-08-07'],
  ['2019-08-11', '2019-08-12'],
  ['2019-08-16', '2019-08-17'],
  ['2020-07-31', '2020-08-01'],
  ['2020-08-05', '2020-08-06'],
  ['2020-08-10', '2020-08-11'],
  ['2021-06-26', '2021-06-27'],
  ['2021-07-01', '2021-07-02'],
  ['2021-07-11', '2021-07-12']],
 'original_data': {'opt_imgs': {'train': ['2019-08-06-2019-08-07.tif',
    '2019-08-11-2019-08-12.tif',
    '2019-08-16-2019-08-17.tif',
    '2020-07-31-2020-08-01.tif',
    '2020-08-05-2020-08-06.tif',
    '2020-08-10-2020-08-11.tif'],
   'test': ['2020-07-31-2020-08-01.tif',
    '2020-08-05-2020-08-06.tif',
    '2020-08-10-2020-08-11.tif',
    '2021-06-26-2021-06-27.tif',
    '2021-07-01-2021-07-02.tif',
    '2021-07-11-2021-07-12.tif']},
  'sar_

# Reading Location data

In [9]:
roi = ee.Geometry.Rectangle([cfg['xmin'], cfg['ymin'], cfg['xmax'], cfg['ymax']])

In [66]:
map_view = geemap.Map(center=(cfg['ycenter'], cfg['xcenter']), zoom=8)
map_view


Map(center=[-7.10765019, -59.98953959], controls=(WidgetControl(options=['position', 'transparent_bg'], widget…

In [67]:
#location = geemap.shp_to_ee(str(location_shape))
map_view.addLayer(roi, {}, 'Location')

# Dates

In [120]:
before_days = cfg['before_days']
d0 = datetime.datetime.strptime(cfg['date'][0], "%Y-%m-%d")
d1 = datetime.datetime.strptime(cfg['date'][1], "%Y-%m-%d")
d2 = datetime.datetime.strptime(cfg['date'][2], "%Y-%m-%d")
dt1 = (d1-d0).days + before_days
dt2 = (d2-d1).days + before_days

In [11]:
start_date = d0 - datetime.timedelta(days=before_days)
end_date = d2 + datetime.timedelta(days=before_days)
print(f'Analyzed Period: {start_date.date()}:{end_date.date()}')


Analyzed Period: 2019-07-15:2021-08-01


In [12]:
opt_vizParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 2048
}

sar_vizParams = {
  'bands': ['VV'],
  'min': -20,
  'max': 1
}

cloud_vizParams = {
  'bands': ['MSK_CLDPRB'],
  'min': 0,
  'max': 100
}

# SAR Data

## Average Data

In [121]:
sar_collection = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT').filterDate(str(start_date.date()), str(end_date.date())).filterBounds(roi)
ds = np.arange(dt1)
img_collections = []
split_ds = np.array_split(ds, cfg['n_sar_avg_images'])
for i, dsi in enumerate(split_ds):
    di0 = d0 - datetime.timedelta(days=before_days) + datetime.timedelta(days=int(dsi[0]))
    di1 = d0 - datetime.timedelta(days=before_days) + datetime.timedelta(days=int(dsi[-1]))
    img_col = sar_collection.filterDate(str(di0.date()), str(di1.date()))
    img = img_col.mean().clip(roi)
    #print(f'{di0.date()}-{di1.date()}-delta:{(di1-di0).days}|{len(img_col.getInfo()["features"])} images')
    print(f'{di0.date()}-{di1.date()}')
    map_view.addLayer(img, sar_vizParams, f't1-{i:02d}')
    #export_sar(img, roi, f'{di0.date()}-{di1.date()}')
    

2019-07-14-2019-08-14
2019-08-15-2019-09-15
2019-09-16-2019-10-17
2019-10-18-2019-11-18
2019-11-19-2019-12-20
2019-12-21-2020-01-21
2020-01-22-2020-02-22
2020-02-23-2020-03-25
2020-03-26-2020-04-26
2020-04-27-2020-05-28
2020-05-29-2020-06-29
2020-06-30-2020-07-30


In [122]:
ds = np.arange(dt2)
img_collections = []
split_ds = np.array_split(ds,  cfg['n_sar_avg_images'])
for i, dsi in enumerate(split_ds):
    di0 = d1 - datetime.timedelta(days=before_days) + datetime.timedelta(days=int(dsi[0]))
    di1 = d1 - datetime.timedelta(days=before_days) + datetime.timedelta(days=int(dsi[-1]))
    img_col = sar_collection.filterDate(str(di0.date()), str(di1.date()))
    img = img_col.mean().clip(roi)
    #print(f'{di0.date()}-{di1.date()}-delta:{(di1-di0).days}|{len(img_col.getInfo()["features"])} images')
    print(f'{di0.date()}-{di1.date()}')
    map_view.addLayer(img, sar_vizParams, f't2-{i:02d}')
    #export_sar(img, roi, f'{di0.date()}-{di1.date()}')

2020-06-30-2020-07-30
2020-07-31-2020-08-30
2020-08-31-2020-09-30
2020-10-01-2020-10-31
2020-11-01-2020-12-01
2020-12-02-2021-01-01
2021-01-02-2021-02-01
2021-02-02-2021-03-03
2021-03-04-2021-04-02
2021-04-03-2021-05-02
2021-05-03-2021-06-01
2021-06-02-2021-07-01


## Multiple Images

In [110]:
date_0, date_1 = d0 - datetime.timedelta(days=12), d1 + datetime.timedelta(days=12)
sar_collection = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT').filterDate(str(date_0.date()), str(date_1.date())).filterBounds(roi)
img_col = sar_collection.filter(ee.Filter.eq('platform_number', 'A'))
imgs = img_col.getInfo()['features']
print(len(imgs))
print(f'{date_0.date()}---{date_1.date()}')
cycles = set({})
for im in imgs:
    cycle = im['properties']['cycleNumber'] 
    cycles.add(cycle)
cycles = list(cycles)
print(len(cycles))
skip_idx = [1, 4]
count = 0
for idx, cycle in enumerate(cycles):
    if idx in skip_idx:
        continue
    cycle_col = sar_collection.filter(ee.Filter.eq('cycleNumber', cycle))
    img = cycle_col.mosaic()
    data_check = img.mask().reduceRegion(ee.Reducer.allNonZero(), roi).getInfo()
    if not 0 in data_check.values():
        count+=1
        img = img.clip(roi)
        #map_view.addLayer(img, sar_vizParams, f't2-{str(cycle)}')
        dates = set({})
        cycle_imgs = cycle_col.getInfo()["features"]
        for cycle_img in cycle_imgs:
            day = datetime.datetime.strptime(cycle_img['properties']['system:index'][17:25], '%Y%m%d')
            dates.add(day)
        dates = list(dates)
        dates.sort()
        print(f'{idx}: {dates[0].date()}---{dates[-1].date()}')
        export_sar(img, roi, f'{dates[0].date()}-{dates[-1].date()}')
print(count)

121
2019-08-02---2020-08-12
32
0: 2019-08-05---2019-08-12
2: 2019-08-29---2019-09-05
3: 2019-09-10---2019-09-17
5: 2019-10-04---2019-10-11
6: 2019-10-16---2019-10-23
7: 2019-10-28---2019-11-04
8: 2019-11-09---2019-11-16
9: 2019-11-21---2019-11-28
11: 2019-12-15---2019-12-22
12: 2019-12-27---2020-01-03
13: 2020-01-08---2020-01-15
14: 2020-01-20---2020-01-27
15: 2020-02-01---2020-02-08
16: 2020-02-13---2020-02-13
17: 2020-02-25---2020-03-03
18: 2020-03-08---2020-03-15
19: 2020-03-20---2020-03-27
20: 2020-04-01---2020-04-08
21: 2020-04-13---2020-04-20
22: 2020-04-25---2020-05-02
23: 2020-05-07---2020-05-07
24: 2020-05-19---2020-05-26
25: 2020-05-31---2020-06-07
26: 2020-06-12---2020-06-19
27: 2020-06-24---2020-07-01
28: 2020-07-06---2020-07-13
29: 2020-07-18---2020-07-25
30: 2020-07-30---2020-08-06
31: 2020-08-11---2020-08-11
29


In [111]:
date_0, date_1 = d1 - datetime.timedelta(days=12), d2 + datetime.timedelta(days=12)
sar_collection = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT').filterDate(str(date_0.date()), str(date_1.date())).filterBounds(roi)
img_col = sar_collection.filter(ee.Filter.eq('platform_number', 'A'))
imgs = img_col.getInfo()['features']
print(len(imgs))
print(f'{date_0.date()}---{date_1.date()}')
cycles = set({})
for im in imgs:
    cycle = im['properties']['cycleNumber'] 
    cycles.add(cycle)
cycles = list(cycles)
print(len(cycles))
skip_idx = []
count = 0
for idx, cycle in enumerate(cycles):
    if idx in skip_idx:
        continue
    cycle_col = sar_collection.filter(ee.Filter.eq('cycleNumber', cycle))
    img = cycle_col.mosaic()
    data_check = img.mask().reduceRegion(ee.Reducer.allNonZero(), roi).getInfo()
    if not 0 in data_check.values():
        count+=1
        img = img.clip(roi)
        #map_view.addLayer(img, sar_vizParams, f't2-{str(cycle)}')
        dates = set({})
        cycle_imgs = cycle_col.getInfo()["features"]
        for cycle_img in cycle_imgs:
            day = datetime.datetime.strptime(cycle_img['properties']['system:index'][17:25], '%Y%m%d')
            dates.add(day)
        dates = list(dates)
        dates.sort()
        print(f'{idx}: {dates[0].date()}---{dates[-1].date()}')
        export_sar(img, roi, f'{dates[0].date()}-{dates[-1].date()}')
print(count)

118
2020-07-19---2021-07-14
31
1: 2020-07-30---2020-08-06
2: 2020-08-11---2020-08-18
3: 2020-08-23---2020-08-30
4: 2020-09-04---2020-09-11
5: 2020-09-16---2020-09-23
6: 2020-09-28---2020-10-05
7: 2020-10-10---2020-10-17
8: 2020-10-22---2020-10-29
9: 2020-11-03---2020-11-10
10: 2020-11-15---2020-11-22
12: 2020-12-09---2020-12-16
13: 2020-12-21---2020-12-28
14: 2021-01-02---2021-01-09
15: 2021-01-14---2021-01-21
16: 2021-01-26---2021-02-02
17: 2021-02-07---2021-02-14
18: 2021-02-19---2021-02-26
19: 2021-03-03---2021-03-10
20: 2021-03-15---2021-03-22
21: 2021-03-27---2021-04-03
22: 2021-04-08---2021-04-15
23: 2021-04-20---2021-04-27
24: 2021-05-02---2021-05-09
25: 2021-05-14---2021-05-21
26: 2021-05-26---2021-06-02
27: 2021-06-07---2021-06-14
28: 2021-06-19---2021-06-26
29: 2021-07-01---2021-07-08
30: 2021-07-13---2021-07-13
29


# Optical Data (Level 2A)

In [13]:
opt_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate(start_date, end_date)
opt_collection = opt_collection.filterBounds(roi)
opt_info = opt_collection.getInfo()

In [28]:
for info in opt_info['features']:
    img_date = datetime.datetime.strptime(info['properties']['DATATAKE_IDENTIFIER'][5:13], '%Y%m%d')
    img_lct = info['properties']['PRODUCT_ID']
    cloud = info['properties']['CLOUDY_PIXEL_PERCENTAGE']
    print(img_date, img_lct, cloud)
    #break

2019-07-17 00:00:00 S2B_MSIL2A_20190717T142049_N0213_R010_T20MQS_20190717T193145 13.581067
2019-07-17 00:00:00 S2B_MSIL2A_20190717T142049_N0213_R010_T20MQT_20190717T193145 2.131074
2019-07-17 00:00:00 S2B_MSIL2A_20190717T142049_N0213_R010_T20MRS_20190717T193145 2.200809
2019-07-17 00:00:00 S2B_MSIL2A_20190717T142049_N0213_R010_T20MRT_20190717T193145 0.461807
2019-07-17 00:00:00 S2B_MSIL2A_20190717T142049_N0213_R010_T21MTM_20190717T193145 0.595476
2019-07-17 00:00:00 S2B_MSIL2A_20190717T142049_N0213_R010_T21MTN_20190717T193145 0.506153
2019-07-22 00:00:00 S2A_MSIL2A_20190722T142041_N0213_R010_T20MQS_20190722T164016 0.299776
2019-07-22 00:00:00 S2A_MSIL2A_20190722T142041_N0213_R010_T20MQT_20190722T164016 0.125833
2019-07-22 00:00:00 S2A_MSIL2A_20190722T142041_N0213_R010_T20MRS_20190722T164016 0.02207
2019-07-22 00:00:00 S2A_MSIL2A_20190722T142041_N0213_R010_T20MRT_20190722T164016 0.167836
2019-07-22 00:00:00 S2A_MSIL2A_20190722T142041_N0213_R010_T21MTM_20190722T164016 0.001543
2019-07-22

In [29]:
for date_i in cfg['opt_images_dates']:
    opt_d0 = date_i[0]
    opt_d1 = date_i[1]
    opt_collection_i = opt_collection.filterDate(opt_d0, opt_d1)
    opt_mosaic_i = opt_collection_i.mosaic()
    export_opt(opt_mosaic_i, roi, f'{opt_d0}-{opt_d1}')
    export_maskcloud(opt_mosaic_i, roi, f'cloud_{opt_d0}-{opt_d1}')
    map_view.addLayer(opt_mosaic_i.clip(roi), opt_vizParams, f'opt-{opt_d0}-{opt_d1}')
    print({opt_d0}-{opt_d1})
    print(f'{len(opt_collection_i.getInfo()["features"])} images')


    

{'2019-08-06'}
6 images
{'2019-11-24'}
6 images
{'2020-02-17'}
6 images
{'2020-05-02'}
6 images
{'2020-07-31'}
6 images


# Optical Data (Level 1C)

In [23]:
opt_collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate(start_date, end_date)
opt_collection = opt_collection.filterBounds(roi)
opt_info = opt_collection.getInfo()

In [19]:
for info in opt_info['features']:
    img_date = datetime.datetime.strptime(info['properties']['DATATAKE_IDENTIFIER'][5:13], '%Y%m%d')
    img_lct = info['properties']['PRODUCT_ID']
    cloud = info['properties']['CLOUDY_PIXEL_PERCENTAGE']
    print(img_date, img_lct, cloud)
    #break

2019-07-17 00:00:00 S2B_MSIL1C_20190717T142049_N0208_R010_T20MQS_20190717T190327 8.6901
2019-07-17 00:00:00 S2B_MSIL1C_20190717T142049_N0208_R010_T20MQT_20190717T190327 3.0042
2019-07-17 00:00:00 S2B_MSIL1C_20190717T142049_N0208_R010_T20MRS_20190717T190327 0.2135
2019-07-17 00:00:00 S2B_MSIL1C_20190717T142049_N0208_R010_T20MRT_20190717T190327 0.1135
2019-07-17 00:00:00 S2B_MSIL1C_20190717T142049_N0208_R010_T21MTM_20190717T190327 0.0553
2019-07-17 00:00:00 S2B_MSIL1C_20190717T142049_N0208_R010_T21MTN_20190717T190327 0.233
2019-07-22 00:00:00 S2A_MSIL1C_20190722T142041_N0208_R010_T20MQS_20190722T160036 0.0154
2019-07-22 00:00:00 S2A_MSIL1C_20190722T142041_N0208_R010_T20MQT_20190722T160036 0.0185
2019-07-22 00:00:00 S2A_MSIL1C_20190722T142041_N0208_R010_T20MRS_20190722T160036 0
2019-07-22 00:00:00 S2A_MSIL1C_20190722T142041_N0208_R010_T20MRT_20190722T160036 0.0251
2019-07-22 00:00:00 S2A_MSIL1C_20190722T142041_N0208_R010_T21MTM_20190722T160036 0
2019-07-22 00:00:00 S2A_MSIL1C_20190722T142

In [25]:
for date_i in cfg['opt_images_dates']:
    opt_d0 = date_i[0]
    opt_d1 = date_i[1]
    opt_collection_i = opt_collection.filterDate(opt_d0, opt_d1)
    opt_mosaic_i = opt_collection_i.mosaic()
    #export_opt_l1(opt_mosaic_i, roi, f'{opt_d0}-{opt_d1}')
    map_view.addLayer(opt_mosaic_i.clip(roi), opt_vizParams, f'opt-{opt_d0}-{opt_d1}')
    print(f'{opt_d0}-{opt_d1}.tif')
    print(f'{len(opt_collection_i.getInfo()["features"])} images')    

2019-08-06-2019-08-07.tif
6 images
2019-08-11-2019-08-12.tif
6 images
2019-08-16-2019-08-17.tif
6 images
2020-07-31-2020-08-01.tif
6 images
2020-08-05-2020-08-06.tif
6 images
2020-08-10-2020-08-11.tif
6 images
2021-06-26-2021-06-27.tif
6 images
2021-07-01-2021-07-02.tif
6 images
2021-07-11-2021-07-12.tif
6 images
